In [2]:
import ee
import geemap
import json
import geojson
from IPython.display import Image

import os
import pandas as pd
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON, Marker, MarkerCluster

In [3]:
Map = geemap.Map(center=[23.834063, 89.398476], zoom=7)
Map

Map(center=[23.834063, 89.398476], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [4]:
polygon = ee.Geometry.Polygon([
  [[89.395747, 23.845642],
    [89.395747, 23.848162],
    [89.398502, 23.848162],
    [89.398502, 23.845642],
    [89.395747, 23.845642]]
])

In [5]:
polygon.getInfo()

{'type': 'Polygon',
 'coordinates': [[[89.395747, 23.845642],
   [89.398502, 23.845642],
   [89.398502, 23.848162],
   [89.395747, 23.848162],
   [89.395747, 23.845642]]]}

In [6]:
type(polygon)

ee.geometry.Geometry

In [68]:
polygon.getInfo()['coordinates']

[[[89.395747, 23.845642],
  [89.398502, 23.845642],
  [89.398502, 23.848162],
  [89.395747, 23.848162],
  [89.395747, 23.845642]]]

In [69]:
S2_SR = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(polygon).filterDate('2019-01-01', '2020-01-01')

In [72]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

In [73]:
S2_NDVI = S2_SR.map(addNDVI)

In [74]:
recent_S2 = ee.Image(S2_NDVI.sort('system:time_start', False).first())

In [75]:
NDVIpalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']

In [76]:
Map.addLayer(recent_S2.select('NDVI'), {'palette': NDVIpalette}, 'Recent Sentinel NDVI')

In [13]:
task = ee.batch.Export.image.toDrive(**{
'image': recent_S2.select('NDVI'),
'description': 'test_export',
'scale': 5,
'region': polygon.getInfo()['coordinates'],
'crs':'EPSG:4326',
'fileFormat':'GeoTIFF',
})
task.start()

In [14]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
Polling for task (id: ZYYOBINC5U67ZJCFS7S7ZZOG).
